In [45]:
import numpy as np

In [46]:
sequence_length = 2
number_states = 2
mutation_rate = 0.3

In [47]:
# create every possible sequence
sequences = np.array(np.meshgrid(*[np.arange(number_states)]*sequence_length)).T.reshape(-1, sequence_length)
print(sequences)

[[0 0]
 [0 1]
 [1 0]
 [1 1]]


In [48]:
# initial frequencies is vector of zeros and 1 at the first sequence
initial_frequencies = np.zeros(sequences.shape[0])
initial_frequencies[0] = 1
print(initial_frequencies)

[1. 0. 0. 0.]


In [49]:
# initial transition matrix
transition_matrix = np.zeros((sequences.shape[0], sequences.shape[0]))
# fill the transition matrix with the number of different elements in the sequences
for i in range(sequences.shape[0]):
    for j in range(sequences.shape[0]):
        transition_matrix[i, j] = np.sum(sequences[i] != sequences[j])

print(transition_matrix)

[[0. 1. 1. 2.]
 [1. 0. 2. 1.]
 [1. 2. 0. 1.]
 [2. 1. 1. 0.]]


In [50]:
# element-wise apply (mu^c)*(1-mu)^(l-c) to the transition matrix
transition_matrix = ((mutation_rate/(number_states-1))**transition_matrix)*(1-mutation_rate)**(sequence_length-transition_matrix)
print(transition_matrix)
# check sums of rows
print(np.sum(transition_matrix, axis=0))

[[0.49 0.21 0.21 0.09]
 [0.21 0.49 0.09 0.21]
 [0.21 0.09 0.49 0.21]
 [0.09 0.21 0.21 0.49]]
[1. 1. 1. 1.]


In [51]:
frequencies_2 = np.dot(initial_frequencies.T, transition_matrix)
print(frequencies_2)
# check sum of frequencies
print(np.sum(frequencies_2))

[0.49 0.21 0.21 0.09]
0.9999999999999999


In [52]:
frequencies_3 = np.dot(frequencies_2.T, transition_matrix)
print(frequencies_3)
# check sum of frequencies
print(np.sum(frequencies_3))

[0.3364 0.2436 0.2436 0.1764]
0.9999999999999998
